# Multi-Provider Agent System with Guardrails

This notebook demonstrates advanced agent orchestration patterns:
1. **Multi-provider integration**: Using OpenAI, DeepSeek, Gemini, and Groq models within a single workflow
2. **Structured outputs**: Enforcing type-safe agent responses using Pydantic models
3. **Input guardrails**: Implementing pre-execution validation to prevent undesired behavior

In [ ]:
# Import dependencies
from dotenv import load_dotenv
from openai import AsyncOpenAI
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel, input_guardrail, GuardrailFunctionOutput
from typing import Dict
import os
from pydantic import BaseModel

In [ ]:
# Initialize Environment
load_dotenv(override=True)

In [ ]:
# Configure Multi-Provider Clients
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
DEEPSEEK_BASE_URL = "https://api.deepseek.com/v1"
GROQ_BASE_URL = "https://api.groq.com/openai/v1"

deepseek_client = AsyncOpenAI(base_url=DEEPSEEK_BASE_URL, api_key=os.getenv('DEEPSEEK_API_KEY'))
gemini_client = AsyncOpenAI(base_url=GEMINI_BASE_URL, api_key=os.getenv('GOOGLE_API_KEY'))
groq_client = AsyncOpenAI(base_url=GROQ_BASE_URL, api_key=os.getenv('GROQ_API_KEY'))

deepseek_model = OpenAIChatCompletionsModel(model="deepseek-chat", openai_client=deepseek_client)
gemini_model = OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=gemini_client)
llama3_3_model = OpenAIChatCompletionsModel(model="llama-3.3-70b-versatile", openai_client=groq_client)

In [ ]:
# Define Multi-Model Agents
instructions = "You are a sales agent for ComplAI, a SOC2 compliance SaaS platform."

sales_agent_deepseek = Agent(name="DeepSeek Agent", instructions=instructions, model=deepseek_model)
sales_agent_gemini = Agent(name="Gemini Agent", instructions=instructions, model=gemini_model)
sales_agent_llama = Agent(name="Llama Agent", instructions=instructions, model=llama3_3_model)

# Convert to tools for orchestration
tool1 = sales_agent_deepseek.as_tool(tool_name="deepseek_agent", tool_description="Generate sales email")
tool2 = sales_agent_gemini.as_tool(tool_name="gemini_agent", tool_description="Generate sales email")
tool3 = sales_agent_llama.as_tool(tool_name="llama_agent", tool_description="Generate sales email")

In [ ]:
# Define Orchestrator Agent
manager_instructions = """
You are a Sales Manager. Generate three email drafts using the available agent tools, 
select the best one, and hand off to the Email Manager for delivery.
"""

sales_manager = Agent(
    name="Sales Manager",
    instructions=manager_instructions,
    tools=[tool1, tool2, tool3],
    model="gpt-4o-mini"
)

In [ ]:
# Execute Workflow
async def run_multi_provider_workflow():
    message = "Generate a cold sales email"
    with trace("Multi-Provider Workflow"):
        result = await Runner.run(sales_manager, message)
        print(result.final_output)

await run_multi_provider_workflow()

## Structured Outputs with Pydantic

Ensuring type-safe, parseable agent responses.

In [ ]:
# Define Output Schema
class NameCheckOutput(BaseModel):
    is_name_in_message: bool
    name: str

# Agent with Structured Output
guardrail_agent = Agent(
    name="Name Validator",
    instructions="Detect if a personal name is mentioned in the user's message.",
    output_type=NameCheckOutput,
    model="gpt-4o-mini"
)

In [ ]:
# Implement Input Guardrail
@input_guardrail
async def guardrail_against_name(ctx, agent, message):
    """Validates that no personal names are included in the request."""
    result = await Runner.run(guardrail_agent, message, context=ctx.context)
    is_name_in_message = result.final_output.is_name_in_message
    return GuardrailFunctionOutput(
        output_info={"found_name": result.final_output},
        tripwire_triggered=is_name_in_message
    )

In [ ]:
# Apply Guardrail to Agent
protected_sales_manager = Agent(
    name="Protected Sales Manager",
    instructions=manager_instructions,
    tools=[tool1, tool2, tool3],
    model="gpt-4o-mini",
    input_guardrails=[guardrail_against_name]
)

In [ ]:
# Test Guardrail: Should Trigger
async def test_guardrail_trigger():
    message = "Send a cold sales email from Alice"
    with trace("Guardrail Test - Trigger"):
        try:
            result = await Runner.run(protected_sales_manager, message)
            print(result.final_output)
        except Exception as e:
            print(f"Guardrail blocked execution: {e}")

await test_guardrail_trigger()

In [ ]:
# Test Guardrail: Should Pass
async def test_guardrail_pass():
    message = "Send a cold sales email from the Head of Business Development"
    with trace("Guardrail Test - Pass"):
        result = await Runner.run(protected_sales_manager, message)
        print(result.final_output)

await test_guardrail_pass()